In [1]:
import torch as t
from torch import nn
from torch.utils.data import Dataset, DataLoader
import plotly.express as px
from IPython.display import display
import pandas as pd
import numpy as np
import copy
from fancy_einsum import einsum
from dataclasses import dataclass
from tqdm.notebook import tqdm_notebook

from einops import rearrange, reduce, repeat

import utils
import cnn_modules as cm
import transformer_modules as tm

In [2]:
def single_head_attention(Q: t.Tensor, K: t.Tensor, V: t.Tensor) -> t.Tensor:
    '''
    Should return the results of self-attention (see the "Self-Attention in Detail" section of the Illustrated Transformer).

    With this function, you can ignore masking.

    Q: shape (batch, seq_len, embed_size)
    K: shape (batch, seq_len, embed_size)
    V: shape (batch, seq_len, embed_size)

    Return: shape (batch, seq_len, embed_size)
    '''
    scores = Q @ t.transpose(K, -2, -1)
    scores /= Q.shape[-1] ** 0.5
    scores = t.softmax(scores, dim=-1)
    Z = einsum('B Seq Score, B Seq Emb -> B Seq Emb', scores, V)
    return Z
    

In [3]:
def single_head_masked_attention(Q: t.Tensor, K: t.Tensor, V: t.Tensor) -> t.Tensor:
    '''
    Should return the results of masked self-attention.

    See "The Decoder Side" section of the Illustrated Transformer for an explanation of masking.

    Q: shape (batch, seq_len, embed_size)
    K: shape (batch, seq_len, embed_size)
    V: shape (batch, seq_len, embed_size)

    Return: shape (batch, seq_len, embed_size)
    '''
    batch_size, seq_len, embed_size = Q.shape
    scores = Q @ t.transpose(K, -2, -1)
    scores /= Q.shape[-1] ** 0.5
    
    # create lower-left triangle of ones, including the diagonal
    mask = t.tril(t.ones(seq_len, seq_len), diagonal=0)
    # fill with close-to-neg-inf values
    scores = scores.masked_fill(mask==0, -1e9)
    
    scores = t.softmax(scores, dim=-1)
    Z = einsum('B Seq Score, B Seq Emb -> B Seq Emb', scores, V)
    return Z
    

In [4]:
def multihead_masked_attention(Q: t.Tensor, K: t.Tensor, V: t.Tensor, num_heads: int):
    '''
    Implements multihead masked attention on the matrices Q, K and V.

    Q: shape (batch, seq, nheads*headsize)
    K: shape (batch, seq, nheads*headsize)
    V: shape (batch, seq, nheads*headsize)

    returns: shape (batch, seq, nheads*headsize)
    '''
    Q = rearrange(Q, 'B S (nheads headsize) -> B S nheads headsize', nheads = num_heads)
    K = rearrange(K, 'B S (nheads headsize) -> B S nheads headsize', nheads = num_heads)
    V = rearrange(V, 'B S (nheads headsize) -> B S nheads headsize', nheads = num_heads)

    batch_size, seq_len, nheads, headsize = Q.shape
    scores = einsum('B Qseq nheads headsize, B Kseq nheads headsize -> B nheads Qseq Kseq', Q, K)
    scores /= Q.shape[-1] ** 0.5

    # create lower-left triangle of ones, including the diagonal
    mask = t.tril(t.ones(seq_len, seq_len), diagonal=0)
    # fill with close-to-neg-inf values where mask==0
    scores = scores.masked_fill(mask==0, -1e9)

    scores = t.softmax(scores, dim=-1)
    Z = einsum('B nheads Qseq Kseq, B Qseq nheads headsize -> B Qseq nheads headsize', scores, V)
    Z = rearrange(Z, 'B Qseq nheads headsize -> B Qseq (nheads headsize)')
    return Z

In [5]:
T = t.randn(2, 10, 4)

In [6]:
multihead_masked_attention(T, T, T, 2)

tensor([[[ 0.0690, -0.1922,  0.2547,  0.1276],
         [-1.0280,  0.3918,  0.6597,  0.2924],
         [ 2.6227, -0.2859,  0.4566, -0.8573],
         [ 0.0169,  1.2004,  0.2267,  0.0525],
         [-0.4390, -0.0982,  0.2922,  1.3428],
         [ 2.5959,  0.8279,  0.3797,  0.9342],
         [ 1.6417, -0.4528, -0.4696, -2.2922],
         [ 0.2549,  0.7094, -0.3652, -0.3461],
         [-0.4314, -1.0372, -0.1216,  0.2004],
         [-1.4438,  1.1699, -0.3740,  0.8607]],

        [[ 1.4220,  0.3436,  0.0193,  0.6921],
         [-1.1452,  0.3468,  0.3634, -0.5552],
         [ 0.7866,  0.7927,  1.0079, -0.0143],
         [ 1.7841,  0.2455,  0.3745, -0.0996],
         [ 0.0591, -0.1512,  0.7890,  0.2173],
         [ 0.4247, -0.6400,  0.4725, -0.9128],
         [ 0.0356, -0.3364, -0.5728, -0.6655],
         [ 0.1734, -1.0741, -1.4653, -0.0248],
         [-0.0344, -0.1084,  1.0520, -1.0145],
         [-0.4470, -1.5919,  0.0444, -1.1388]]])

In [7]:
class MultiheadMaskedAttention(nn.Module):
    W_QKV: nn.Linear
    W_O: nn.Linear

    def __init__(self, hidden_size: int, num_heads: int):
        super().__init__()
        self.num_heads = num_heads
        self.query_size = int(hidden_size / num_heads)
        self.qkv = cm.Linear(hidden_size, 3*hidden_size)
        self.ff = cm.Linear(hidden_size, hidden_size)

    def multihead_masked_attention(self, Q: t.Tensor, K: t.Tensor, V: t.Tensor, num_heads: int):
        '''
        Implements multihead masked attention on the matrices Q, K and V.

        Q: shape (batch, seq, nheads*headsize)
        K: shape (batch, seq, nheads*headsize)
        V: shape (batch, seq, nheads*headsize)

        returns: shape (batch, seq, nheads*headsize)
        '''
        Q = rearrange(Q, 'B S (nheads headsize) -> B S nheads headsize', nheads = num_heads)
        K = rearrange(K, 'B S (nheads headsize) -> B S nheads headsize', nheads = num_heads)
        V = rearrange(V, 'B S (nheads headsize) -> B S nheads headsize', nheads = num_heads)

        batch_size, seq_len, nheads, headsize = Q.shape
        scores = einsum('B Qseq nheads headsize, B Kseq nheads headsize -> B nheads Qseq Kseq', Q, K)
        scores /= Q.shape[-1] ** 0.5

        # create lower-left triangle of ones, including the diagonal
        mask = t.tril(t.ones(seq_len, seq_len), diagonal=0)
        # fill with close-to-neg-inf values where mask==0
        scores = scores.masked_fill(mask==0, -1e9)

        scores = t.softmax(scores, dim=-1)
        Z = einsum('B nheads Qseq Kseq, B Qseq nheads headsize -> B Qseq nheads headsize', scores, V)
        Z = rearrange(Z, 'B Qseq nheads headsize -> B Qseq (nheads headsize)')
        return Z

    def forward(self, x: t.Tensor) -> t.Tensor:
        '''
        x: shape (batch, seq, hidden_size)

        Return: shape (batch, seq, hidden_size)
        '''
        out = self.qkv(x)
        Q, K, V = t.tensor_split(out, 3, dim=-1)

        Z = self.multihead_masked_attention(Q, K, V, self.num_heads)

        return self.ff(Z)

In [8]:
@dataclass(frozen=True)
class TransformerConfig:
    '''Constants used throughout your decoder-only transformer model.'''

    num_layers: int
    num_heads: int
    vocab_size: int
    hidden_size: int
    max_seq_len: int
    dropout: float = 0.1
    layer_norm_epsilon: float = 1e-05

In [9]:
config = TransformerConfig(
    num_layers=4, 
    num_heads=2, 
    vocab_size=500, 
    hidden_size=64,
    max_seq_len=100,
    dropout=0.1)

In [10]:
config.hidden_size

64

In [11]:
class MLP(nn.Module):

    def __init__(self, hidden_size, dropout):
        super().__init__()
        self.linear1 = cm.Linear(hidden_size, 4 * hidden_size)
        self.gelu = tm.GELU()
        self.linear2 = cm.Linear(4 * hidden_size, hidden_size)
        self.dropout = tm.Dropout(p=dropout)

    def forward(self, x):
        out = self.gelu(self.linear1(x))
        out = self.dropout(self.linear2(out))
        return out

In [12]:
class DecoderBlock(nn.Module):

    def __init__(self, config: TransformerConfig):
        super().__init__()
        self.attn = tm.MultiheadMaskedAttention(config.hidden_size, config.num_heads)
        self.lnorm1 = tm.LayerNorm(config.hidden_size, eps=config.layer_norm_epsilon)
        self.mlp = tm.MLP(config.hidden_size, config.dropout)
        self.lnorm2 = tm.LayerNorm(config.hidden_size, eps=config.layer_norm_epsilon)

    def forward(self, x: t.Tensor) -> t.Tensor:
        normed_attn = self.lnorm1(self.attn(x))
        out = normed_attn + x
        normed_mlp = self.lnorm2(self.mlp(out))
        out = normed_mlp + out
        return out


In [13]:
class DecoderOnlyTransformer(nn.Module):

    def __init__(self, config: TransformerConfig):
        super().__init__()
        self.emb = tm.Embedding(config.vocab_size, config.hidden_size)
        self.pos_enc = tm.PositionalEncoding(config.max_seq_len, config.hidden_size)
        self.dropout = tm.Dropout(p=config.dropout)

        decoders = [DecoderBlock(config) for l in range(config.num_layers)]
        self.decoders = nn.Sequential(*decoders)
        
        self.post_norm = tm.LayerNorm(config.hidden_size)

    def forward(self, x: t.Tensor) -> t.Tensor:
        embedding = self.emb(x)
        embedding = self.pos_enc(embedding)
        embedding = self.dropout(embedding)

        out = self.decoders(embedding)
        out = self.post_norm(out)

        out = einsum("B S E, V E -> B S V", out, self.emb.weight)

        return out

In [14]:
device = t.device("cuda:0" if t.cuda.is_available() else "cpu")